<a href="https://colab.research.google.com/github/Eddy109109/Dacon_Competition/blob/master/20210310_%EA%B5%AD%ED%9A%8C%EC%9E%85%EB%B2%951.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#라이브러리


In [ ]:
!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

+ pip install konlpy
     |████████████████████████████████| 19.4MB 1.8MB/s 
     |████████████████████████████████| 460kB 46.4MB/s 
     |████████████████████████████████| 92kB 8.9MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
+ bash -x
+ curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh
+ mecab_dicdir=/usr/local/lib/mecab/dic/mecab-ko-dic
+ set -e
++ uname
+ os=Linux
+ [[ ! Linux == \L\i\n\u\x ]]
+ hash sudo
+ sudo=sudo
+ python=python3
+ hash pyenv
+ at_user_site=
++ check_python_site_location_is_writable
++ python3 -
+ [[ 1 == \0 ]]
+ hash automake
+ echo 'Installing automake (A dependency for mecab-ko)'
Installing automake (A dependency for mecab-ko)
+ install_automake
+ '[' Linux == Linux ']'
++ grep -Ei 'debian|buntu|mint' /etc/lsb-release /etc/os-release
+ '[' '/etc/lsb-release:DISTRIB_ID=Ubuntu
/etc/lsb-release:DISTRIB_DESCRIPTION="Ubuntu 18.04.5 

In [ ]:
!git clone https://github.com/facebookresearch/fastText.git
!cd fastText
!pip install fastText
import fasttext.util

Cloning into 'fastText'...
remote: Enumerating objects: 3854, done.
remote: Total 3854 (delta 0), reused 0 (delta 0), pack-reused 3854
Receiving objects: 100% (3854/3854), 8.22 MiB | 31.43 MiB/s, done.
Resolving deltas: 100% (2417/2417), done.
     |████████████████████████████████| 71kB 3.4MB/s 
  Created wheel for fastText: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3093731 sha256=3db9ef9f681c799dbd8c255b308ed7fea5fc2ec0d08cf845801236c8c82b628b
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fastText


In [ ]:
!pip install selenium

     |████████████████████████████████| 911kB 5.3MB/s 


In [ ]:
import re
import json
import time
import konlpy
import gensim 
import glob, os
import requests
import fasttext
import numpy as np
import pandas as pd
import seaborn as sns
from glob import glob
import urllib.request
import matplotlib as mpl
from matplotlib import rc
from konlpy.tag import Mecab
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from collections import Counter
import matplotlib.pyplot as plt
from PIL import Image, ImageFilter
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import matplotlib.font_manager as fm
from matplotlib import font_manager, rc
from wordcloud import ImageColorGenerator
from IPython.display import Image, display
from sklearn.metrics import silhouette_score
from selenium.webdriver.common.keys import Keys
from gensim.models import Word2Vec, FastText, Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud, STOPWORDS as stopwords
from selenium.webdriver.common.action_chains import ActionChains
%matplotlib inline
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)

#함수모음


In [ ]:
mecab = Mecab()
def tokenized_mecab(data):
    result = mecab.morphs(data)
    result = ' '.join(result)
    return result

In [ ]:
def out_stopwords(data, list_stopwords):
    data = data.split(" ")
    list_stopwords = list_stopwords
    data = [token for token in data if token not in list_stopwords]
    return data

In [ ]:
def show_bills(bills_id, df_bills):
    for bill_id in bills_id:
        print(df_bills[df_bills['bill_id']==int(bill_id)]['법안명_법안내용'].iloc[0])
        print("*"*80)        
        
def input2vec(user_input, stopword_list, word2vec_word_dict):
    tokenized_input = tokenized_mecab(user_input)
    tokenized_input_outStopwords = out_stopwords(tokenized_input, stopword_list)
    
    list_vector = []
    for word in tokenized_input_outStopwords:
        if word in word_dict.keys():
            list_vector.append(word_dict[word])
    user_vector = (np.sum(list_vector, axis=0) / len(list_vector)).tolist()
    return user_vector

def similar_senators(user_vector, df_snt):
    similarity = {}
    for idx in df_snt.index:
        if df_snt.loc[idx]['vector'] != '0.0':
            sim = cosine_similarity(np.array(user_vector).reshape(1,-1), np.array([float(i) for i in df_snt.loc[idx]['vector'][1:-1].split(', ')]).reshape(1,-1))
        similarity[str(df_snt.loc[idx]['id'])] = float(sim)
    similarity = {key: value for key, value in sorted(similarity.items(), key=lambda item: item[1], reverse=True)}
    rating = [key for key, value in sorted(similarity.items(), key=lambda item: item[1], reverse=True)]
    top_5 = rating[:5]
    result = {}
    for i in top_5:
        result[i] = str(abs(round((similarity[i]*100), 2))) + "%"
        
    return result

def similar_bills(user_vector, df_bills, df_snt, fasttext_word_dict, snt_ids):
    senators = {}
    for id_ in tqdm(snt_ids):
        id_ = int(id_)
        NAME_HAN = df_snt[df_snt['id']==id_]['NAME_HAN'].iloc[0]
        bills_snt_ids = df_bills[df_bills['senators_list'].str.contains(NAME_HAN)]['BILL_ID'].values
        
        similarity = {}
        for bill_id in bills_snt_ids:
            sim = cosine_similarity(np.array(user_vector).reshape(1,-1), np.array([float(i) for i in df_bills[df_bills['BILL_ID']==bill_id].iloc[0]['vector'][1:-1].split(', ')]).reshape(1,-1))
            similarity[df_bills[df_bills['BILL_ID']==bill_id]['BILL_ID'].iloc[0]] = float(sim)
        similarity = {key: value for key, value in sorted(similarity.items(), key=lambda item: item[1], reverse=True)}
        rating = [str(key) for key, value in sorted(similarity.items(), key=lambda item: item[1], reverse=True)]
        top_5 = rating[:3]
        result = {}
        for i in top_5:
            result[i] = str(abs(round((similarity[i]*100), 2))) + "%"
        senators[id_] = result
    
    return senators

def show_bills(senators_id_bills, df_bills, df_snt):
    result = []
    
    for snt_id in senators_id_bills:
        name = df_snt[df_snt['id'] == snt_id]['NAME'].iloc[0]
        for bill_id in list(senators_id_bills[snt_id]):
            dict_temp = {}
            dict_temp['발의자'] = name
            dict_temp['법안명'] = df_bills[df_bills['BILL_ID']==bill_id]['BILL_NAME'].iloc[0]
            dict_temp['법안내용'] = df_bills[df_bills['BILL_ID']==bill_id]['bill_content'].iloc[0]
            result.append(dict_temp)
    return result

def show_results(df_results, dict_images):
    for snt in df_results['발의자'].unique():
        display(Image('./images/'+snt+'.jpg'))
        print("{} 의원".format(snt))
        df_temp = df_results[df_results['발의자']==snt]
        for idx in df_temp.index:
            print("*"*30, df_temp.loc[idx]['법안명'], "*"*30)
            print(df_temp.loc[idx]['법안내용'], '\n')

# 정보 크롤링

In [ ]:
dict_data = {}
for i in glob('/content/data/*.csv'):
    try:
        dict_data[i.split('/')[3].split('.csv')[0]] = pd.read_csv(i)
    except:
        dict_data[i.split('/')[3].split('.csv')[0]] = pd.read_csv(i, encoding='cp949')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,5,17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
dict_data

##-1 사진


In [ ]:
def download(URL, short_name):
    list_temp = []
    short_name = short_name[0].text.split(' ',1)[-1]
    if short_name not in list_temp:
        list_temp.append(short_name)
        fullName = "/content/images/" + short_name + ".jpg"
        urllib.request.urlretrieve(URL, fullName)   
    else:
        fullName = "/content/images/" + short_name + " (2).jpg"
        urllib.request.urlretrieve(URL, fullName)   

In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

In [ ]:
!apt-get update # to update ubuntu to correctly run apt install

!apt install chromium-chromedriver

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [ ]:
#크롬1
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [ ]:
#크롬2
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: use options instead of chrome_options
  if __name__ == '__main__':


In [ ]:
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


In [ ]:
# 열러라 국회 의원 접속
driver.get('http://watch.peoplepower21.org/?act=&mid=AssemblyMembers&vid=&mode=search&name=&party=&region=&sangim=&gender=&age=&elect_num=')

In [ ]:
start_time = time.time()

for page in range(1,11):
    if page != 1:
        driver.get('http://watch.peoplepower21.org/?mid=AssemblyMembers&mode=search&party=&region=&sangim=&gender=&elect_num=&page=' + str(page))
    for person in range(2,32):
        short_name = driver.find_elements_by_css_selector('#content > div.col-md-8 > div:nth-child('+ str(person)+')> div > a:nth-child(2) > h4')
        image = driver.find_elements_by_tag_name('#content > div.col-md-8 > div:nth-child(' + str(person)+ ') > div > a:nth-child(1) > img')
        src = image[0].get_attribute('src')
        download(src, short_name)
print("---{}초 걸렸슴둥---".format(time.time()-start_time))

---291.6568558216095초 걸렸슴둥---


##-2 법안내용

In [ ]:
sug = dict_data['suggest']

In [54]:
dict_data['suggest'].columns

Index(['BILL_ID', 'BILL_NO', 'BILL_NAME', 'COMMITTEE', 'PROPOSE_DT',
       'PROC_RESULT', 'AGE', 'DETAIL_LINK', 'PROPOSER', 'MEMBER_LIST',
       'RST_PROPOSER', 'PUBL_PROPOSER'],
      dtype='object')

In [ ]:
dict_crawl_data = {}
for idx in tqdm(sug.index):
    bill_id = sug.loc[idx]['BILL_ID']
    url = sug.loc[idx]['DETAIL_LINK']

    res = requests.get(url)

    soup = BeautifulSoup(res.text, 'html.parser')
    container = soup.select('#summaryContentDiv')
    if len(container) == 0:
        dict_crawl_data[bill_id] = '내용없음'
    else:
        contents = container[0].text.strip()
        dict_crawl_data[bill_id] = contents


  0%|          | 280/62658 [06:58<26:37:59,  1.54s/it]

KeyboardInterrupt: ignored

In [58]:
sug['bill_content'] = sug['BILL_ID'].map(dict_crawl_data)

In [59]:
sug.to_csv('/content/data/suggest ver4.0 (members_added')

#Word embedding

##-1 데이터 불러오기 suggest ver4.0

In [60]:
df = pd.read_csv('/content/data/suggest ver4.0 (members_added')

In [61]:
df.columns

Index(['Unnamed: 0', 'BILL_ID', 'BILL_NO', 'BILL_NAME', 'COMMITTEE',
       'PROPOSE_DT', 'PROC_RESULT', 'AGE', 'DETAIL_LINK', 'PROPOSER',
       'MEMBER_LIST', 'RST_PROPOSER', 'PUBL_PROPOSER', 'bill_content'],
      dtype='object')

##word2vec 에 학습할 corpus생성하기

In [78]:
from tqdm import tqdm

In [69]:
from tqdm._tqdm_notebook import tqdm_notebook

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  """Entry point for launching an IPython kernel.


In [71]:
def tqdm_pandas(t):
  from pandas.core.frame import Series
  def inner(series, func, *args, **kwargs):
      t.total = series.size
      def wrapper(*args, **kwargs):
          t.update(1)
          return func(*args, **kwargs)
      result = series.apply(wrapper, *args, **kwargs)
      t.close()
      return result
  Series.progress_apply = inner

In [75]:
tqdm_pandas(tqdm_notebook())

In [79]:
%%time
df['tokenized'] = df['bill_content'].progress_apply(tokenized_mecab)

NotImplementedError: ignored

In [ ]:
#**** stopwords 찾아서 추가
url= "/content/stopwords-ko.txt"    # Stopwords를 정리한 메모장을 불러옴 (약 1,000개의 stopwords)

stopwords_file = list(open(url,"r"))
list_stopword = []
for i in stopwords_file:
    list_stopword.append(i[:-1])

list_stopword.append('제안이유')
list_stopword.append('주요')
list_stopword.append('내용')

In [ ]:
%%time
df['cleared'] = df['tokenized'].progress_apply(lambda x: out_stopwords(x, list_stopwords = list_stopword))

KeyError: ignored

In [55]:
tokenized = ''
for content in df['cleared'].values:
    tokenized = tokenized + "{} \n".format(content)
    
f = open("corpus ver3.0 (전체, stopword 제거).txt", 'w')
f.write(tokenized)
f.close()

KeyError: ignored

In [ ]:
direc ='corpus ver3.0 (전체, stopword 제거).txt'

corpus = [sent.strip().split(" ") for sent in open(direc).readlines()]

##-3 word2vec 학습하기 

In [56]:
 %%time
model_w2v = embedding_model = Word2Vec(corpus, size=100, window = 5, min_count=10,  
                           workers=12, iter=500, hs=0, sg=1)

NameError: ignored

In [57]:
model_w2v.save('models/word2vec ver2.0 (iter500).model')

NameError: ignored

In [ ]:
#모델 불러와서 단어유사도 확인해보기
비슷한 단어들끼리 유사하도록 단어 임베딩이 잘된 것을 확인할 수가 있다.
model_test = Word2Vec.load('models/word2vec ver2.0 (iter500).model')

In [ ]:
model_test.wv.most_similar('성폭행')

In [ ]:
display(Image('./images/bokeh.png'))

In [ ]:
from bokeh.plotting import figure, output_file, save
from bokeh.io import curdoc, show
from bokeh.transform import factor_cmap, factor_mark
from bokeh.models import ColumnDataSource, Grid, LinearAxis, Plot, Text

dict_tsne = dict(x=df.x.tolist(), y=df.y.tolist(), text=df.word.tolist() )
plot_data = ColumnDataSource(dict_tsne)

TOOLS="hover,wheel_zoom,zoom_in,zoom_out"

p = figure(tools=TOOLS, active_scroll='wheel_zoom',
          plot_width= 800,
          plot_height=800)

p.add_tools( HoverTool(tooltips = '@word') )

p.scatter(df['x'], df['y'], fill_alpha=0.6,
          line_color=None)

glyph = Text(x="x", y="y", text="text", angle=0.3, text_color="black")
p.add_glyph(plot_data, glyph)

# output_file("color_scatter.html", title="color_scatter.py example")

show(p);  # open a browser

#법안 벡터 만들기 Sentence2vec